# Library

Required tools for the analysis (could be installed via conda):

- BCFTools
- Genome Analysis Toolkit (GATK) framework

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import rpy2

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
library(rlang)
library(ggplot2)

# Data Collection

## 1. variant/gene - disease association

In [ ]:
! wget -O - https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar_20230903.vcf.gz | gzip -d > clinvar_grch38_20230903.vcf

In [ ]:
# save P/LP variants only

! bcftools view -i 'INFO/CLNSIG ~ "Pathogenic" |INFO/CLNSIG ~ "Likely_pathogenic"' clinvar_grch38_20230903.vcf \
>  clinvar_grch38_20230903_PLP.vcf
! awk '{print "chr"$0}' clinvar_grch38_20230903_PLP.vcf >  clinvar_grch38_20230903_plp.vcf
! sed -i 's/chr#/#/g' clinvar_grch38_20230903_plp.vcf
! rm clinvar_grch38_20230903_PLP.vcf

In [ ]:
gatk VariantsToTable -V clinvar_grch38_20230903_plp.vcf -F CHROM -F POS -F ID -F REF -F ALT -F QUAL -F FILTER -F AF_EXAC -F CLNDN -F CLNDISDB -F CLNREVSTAT -F CLNSIG -F CLNVC -F GENEINFO -O clinvar_grch38_20230903_plp.txt

In [ ]:
cv_plp = pd.read_csv("clinvar_grch38_20230903_plp.txt", sep='\t')

In [ ]:
cv_plp = cv_plp[cv_plp['FILTER'] == "PASS"]
print("N of PLP PASS variants in .vcf:", cv_plp.shape[0])

In [ ]:
# save only gene symbol

cv_plp[['GENEINFO']] = cv_plp[['GENEINFO']].replace(to_replace=':.*', regex=True, value='')

In [ ]:
# rm ids except OMIM and ORPHA
cv_plp[['CLNDISDB']] = cv_plp[['CLNDISDB']].replace(to_replace='MONDO:MONDO:[0-9]*\,', regex=True, value='')
cv_plp[['CLNDISDB']] = cv_plp[['CLNDISDB']].replace(to_replace='MONDO:MONDO:[0-9]*', regex=True, value='')

cv_plp[['CLNDISDB']] = cv_plp[['CLNDISDB']].replace(to_replace='MedGen:[A-Z]*[0-9]*\,', regex=True, value='')
cv_plp[['CLNDISDB']] = cv_plp[['CLNDISDB']].replace(to_replace='MedGen:[A-Z]*[0-9]*', regex=True, value='')

cv_plp[['CLNDISDB']] = cv_plp[['CLNDISDB']].replace(to_replace='MeSH:[A-Z]*[0-9]*\,', regex=True, value='')
cv_plp[['CLNDISDB']] = cv_plp[['CLNDISDB']].replace(to_replace='MeSH:[A-Z]*[0-9]*', regex=True, value='')

cv_plp[['CLNDISDB']] = cv_plp[['CLNDISDB']].replace(to_replace='SNOMED_CT:[0-9]*\,', regex=True, value='')
cv_plp[['CLNDISDB']] = cv_plp[['CLNDISDB']].replace(to_replace='SNOMED_CT:[0-9]*', regex=True, value='')


cv_plp[['CLNDISDB']] = cv_plp[['CLNDISDB']].replace(to_replace='SNOMED_CT:[0-9]*\,', regex=True, value='')
cv_plp[['CLNDISDB']] = cv_plp[['CLNDISDB']].replace(to_replace='SNOMED_CT:[0-9]*', regex=True, value='')

cv_plp[['CLNDISDB']] = cv_plp[['CLNDISDB']].replace(to_replace='Human_Phenotype_Ontology:HP:[0-9]*\,', regex=True, value='')
cv_plp[['CLNDISDB']] = cv_plp[['CLNDISDB']].replace(to_replace='Human_Phenotype_Ontology:HP:[0-9]*', regex=True, value='')

cv_plp[['CLNDISDB']] = cv_plp[['CLNDISDB']].replace(to_replace='\.', regex=True, value='')

In [ ]:
cv_plp.to_csv('clinvar_plp_table.csv', sep='\t')

In [ ]:
# create list of all MIMs
! cat clinvar_plp_table.csv | cut -f11 | sed 's/|/\n/g' | sed 's/\,/\n/g'| grep OMIM | sort | uniq > clinvar_plp_omim.txt
! wc -l clinvar_plp_omim.txt

In [ ]:
disease_omim = pd.read_csv('clinvar_plp_omim.txt', sep='\t', names=['OMIM'], header=None)

In [ ]:
# rm provisional OMIM gene-disease ids (ex, OMIM:PS000000)
disease_omim = disease_omim[disease_omim.OMIM.str.contains('^OMIM:[0-9]{6,6}$', regex= True, na=False)]

In [ ]:
cv_plp["OMIM"] = cv_plp["CLNDISDB"].apply(lambda x: [y for y in disease_omim['OMIM'] if y in x])
cv_plp= cv_plp.explode("OMIM")

In [ ]:
# create list of all MIM - ORPHA links

! cat clinvar_plp_table.csv | cut -f11 | sed 's/|/\n/g' | sort | uniq | grep "^OMIM" | grep Orphanet > disease_mim_orpha.txt

In [ ]:
disease_mim_orpha = pd.read_csv('disease_mim_orpha.txt', sep=',', names=['OMIM', 'ORPHA1','ORPHA2',
                                                                         'ORPHA3','ORPHA4','ORPHA5',
                                                                         'ORPHA6','ORPHA7'])

In [ ]:
disease_omim = disease_omim.merge(disease_mim_orpha, on='OMIM', how='left')

In [ ]:
cv_plp_dis = cv_plp.merge(disease_omim, how='left', on = 'OMIM')

In [ ]:
cv_plp_dis = cv_plp_dis.drop(columns={"CLNDN", "CLNDISDB"})

In [ ]:
cv_plp_dis['ORPHA'] = cv_plp_dis[cv_plp_dis.columns[13:]].apply( lambda x: ','.join(x.dropna().astype(str)),axis=1)

In [ ]:
cv_plp_dis = cv_plp_dis.drop(columns={'ORPHA1', 'ORPHA2','ORPHA3', 'ORPHA4','ORPHA5', 'ORPHA6','ORPHA7'})

In [ ]:
cv_plp_dis.head()

In [ ]:
cv_plp_dis.to_csv("clinvar_plp_dis_ids.csv", sep='\t', index=False)   

# all  PLP PASS variants with associated disease (if present)
# NB! 1 line - variants - disease association

In [ ]:
# ClinVar data statistics

print(cv_plp_dis.shape)
print('N of genes:', cv_plp_dis['GENEINFO'].drop_duplicates().shape[0])
print('N of gene-MIM associations:', cv_plp_dis[['GENEINFO', 'OMIM']].dropna().drop_duplicates(keep='first').shape[0])
print('N of MIMs:', cv_plp_dis[['OMIM']].dropna().drop_duplicates(keep='first').shape[0])

## 2. Phenotype & Inheritance  data 

Terms realted to inheritance
--
- HP:0034345	Mendelian inheritance
- HP:0001426	Non-Mendelian inheritance
- HP:0000006	**Autosomal dominant inheritance**
- HP:0012275	Autosomal dominant inheritance with maternal imprinting
- HP:0000007	**Autosomal recessive inheritance**
- HP:0001417	**X-linked inheritance**
- HP:0001423	**X-linked dominant inheritance**
- HP:0001419	**X-linked recessive inheritance**
- HP:0001450	Y-linked inheritance
- HP:0010984	Digenic inheritance
- HP:0010982	Polygenic inheritance
- HP:0001442	Typified by somatic mosaicism

In [ ]:
hpo_phen_to_gen = pd.read_csv('phenotype_to_genes.txt', sep='\t') # obtained from https://hpo.jax.org/app/
hpo_phen_to_gen.head()

In [ ]:
# select phenotypic features execpt related to inheritance
hpo_pheno = hpo_phen_to_gen.drop(hpo_phen_to_gen[(hpo_phen_to_gen['hpo_id'] == 'HP:0034345') | (hpo_phen_to_gen['hpo_id'] == 'HP:0001426')|
                         (hpo_phen_to_gen['hpo_id'] == 'HP:0000006') | (hpo_phen_to_gen['hpo_id'] == 'HP:0012275') |
                          (hpo_phen_to_gen['hpo_id'] == 'HP:0000007') | (hpo_phen_to_gen['hpo_id'] == 'HP:0001417')|
                          (hpo_phen_to_gen['hpo_id'] == 'HP:0001423') | (hpo_phen_to_gen['hpo_id'] == 'HP:0001419')|
                          (hpo_phen_to_gen['hpo_id'] == 'HP:0001450') | (hpo_phen_to_gen['hpo_id'] == 'HP:0010984')|
                          (hpo_phen_to_gen['hpo_id'] == 'HP:0010982') | (hpo_phen_to_gen['hpo_id'] == 'HP:0001442')].index)

In [ ]:
# collect  terms related to Mendelian inheritance

hpo_inh = hpo_phen_to_gen[(hpo_phen_to_gen['hpo_id'] == 'HP:0000006') | (hpo_phen_to_gen['hpo_id'] == 'HP:0000007') |
                         (hpo_phen_to_gen['hpo_id'] == 'HP:0001417') | (hpo_phen_to_gen['hpo_id'] == 'HP:0001423') |
                          (hpo_phen_to_gen['hpo_id'] == 'HP:0001419')]

In [ ]:
inh_count = hpo_inh.groupby(['gene_symbol', 'disease_id']).size().reset_index(name='count_inhpattern')
hpo_inh = hpo_inh.merge(inh_count, on=['gene_symbol', 'disease_id'])
hpo_inh.head()

In [ ]:
hpo_inh = hpo_inh[hpo_inh['disease_id'] != 'OMIM:268000']  # errors in annotattion due to different causal genes

hpo_inh = hpo_inh.loc[~((hpo_inh['hpo_id']== 'HP:0001417') & (hpo_inh['gene_symbol']== 'CRB1'))] 

In [ ]:
# define inhertance pattern for each gene-disease association

def define_inheritance(df):
    
    if (df['count_inhpattern'] == 1) & (df['hpo_id'] == 'HP:0000006'):
        return 'AD'
    
    elif (df['count_inhpattern'] == 1) & (df['hpo_id'] == 'HP:0000007'):
        return 'AR'
    
    elif (df['hpo_id'] == 'HP:0001423') | (df['hpo_id'] == 'HP:0001419') | (df['hpo_id'] == 'HP:0001417'):
        return 'XL'
    
    elif df['hpo_id'] == 'HP:0001450':
        return 'YL'
    
    else:
        return 'ADAR'

In [ ]:
hpo_inh['Inheritance'] = hpo_inh.apply(define_inheritance, axis=1)

In [ ]:
hpo_inh = hpo_inh.drop(columns=['hpo_id', 'hpo_name', 'count_inhpattern'])
hpo_inh = hpo_inh.drop_duplicates()
hpo_inh.drop_duplicates().shape

In [ ]:
hpo_inh.to_csv('hpo_inh_terms.csv', sep='\t', index=False)

In [ ]:
df = cv_vars.merge(hpo_inh, how="inner", left_on=['GENEINFO', 'OMIM'], right_on=['gene_symbol', 'disease_id'])

In [ ]:
df = df.drop(columns={'disease_id', 'gene_symbol'})
df.head()

In [ ]:
# add ENSEMBL ids

! wget https://www.omim.org/static/omim/data/mim2gene.txt 
! sed -e '1,4d' mim2gene.txt | cut -f4,5 | awk 'NF' > mimgenes.txt
! sed -i 's/Ensembl Gene ID (Ensembl)/Ensembl/g' mimgenes.txt
! rm mim2gene.txt

In [ ]:
ensembl_id = pd.read_csv('mimgenes.txt', sep='\t')
df = df.merge(ensembl_id, left_on=['GENEINFO'], right_on=['Approved Gene Symbol (HGNC)'], how='inner')

In [ ]:
df = df.drop(columns=['Approved Gene Symbol (HGNC)'])
df = df.dropna(subset={'OMIM'})

In [ ]:
# statistics
print(df.shape)
print('N of variants:', df['ID'].drop_duplicates(keep='first').shape[0])
print('N of genes:', df['GENEINFO'].drop_duplicates().shape[0])
print('N of gene-MIM associations:', df[['GENEINFO', 'OMIM']].drop_duplicates(keep='first').shape[0])
print('N of MIMs:', df[['OMIM']].drop_duplicates(keep='first').shape[0])

In [ ]:
# save merged dataframe
df.to_csv('clinvar_hpo_merged_data.csv', sep='\t', index=False)